# EO3 Acid Gas Removal Data Pull

This one presents a number of challenges.  I needed to elimante the 'fast' tags.

Believe I'm still catching some errors for a few tags.  I don't want to accidently write a CSV for a tag that didn't get valid data.

It takes sometihng like 20 hrs to pull a years worth of data.

Let's see if I can write the tag index without any errors?

OK going back to 15 days in January 2020.  None failed.  48 returned empty.  Saved a list of these in 'test'.


In [2]:
import pandas as pd
from functools import reduce
import time

import sys
import clr

sys.path.append(r'C:\Program Files (x86)\PIPC\AF\PublicAssemblies\4.0')
clr.AddReference('OSIsoft.AFSDK')
from OSIsoft.AF import *  
from OSIsoft.AF.PI import *  
from OSIsoft.AF.Asset import *  
from OSIsoft.AF.Data import *  
from OSIsoft.AF.Time import *  
from OSIsoft.AF.UnitsOfMeasure import *

In [15]:
fname = sys.path[0] + '\data\\Geismar_EO3_nameonly.csv'

df_tags = pd.read_csv(fname)

print(df_tags.info())

df_tags.iloc[-875:]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    2413 non-null   object
dtypes: object(1)
memory usage: 19.0+ KB
None


,name
1538,PE32130_EO3
1539,FE32212_EO3_OP
1540,PE31018_EO3
1541,PE31019_EO3
1542,PE31075_EO3
...,...
2408,TE31689_EO3_T3RSMC_EO3_STRH
2409,TE31689_EO3_T3RSMC_EO3_STRL
2410,FE31140S_I3RSMC1P_EO3_VACT
2411,FE31145S_I3RSMC1P_EO3_RECHT


In [16]:
pi_tags = df_tags['name'].iloc[-875:].to_list()
#pi_tags = df_tags['name'].to_list()

print(str(len(pi_tags)) + " pi-tags in the list")


875 pi-tags in the list


In [17]:
# time interval for the PI data
year = '2021'

# SHORT
start_time = year + '-01-1'
end_time = year+ '-12-23'

timerange = AFTimeRange(start_time, end_time)
span = AFTimeSpan.Parse("1m")

# Set the pi server
pi_server = 'DSAMPICOLL'
piServer = PIServers()[pi_server]

#pi_tags[:1]

In [18]:
start = time.perf_counter()
failed_list = []

#df_list = []
#for tag in pi_tags[-6:]:
#for tag in pi_tags[:2]:
for tag in pi_tags:
    tic = time.perf_counter()
    point = PIPoint.FindPIPoint(piServer, tag)  
    name = point.Name.lower()
    #interpolated = point.InterpolatedValues(timerange, span, "", False)  
    try:
        recorded = point.RecordedValues(timerange, AFBoundaryType.Inside, "", False)
        df = pd.DataFrame(recorded.GetValueArrays(None, None, None)[1:3]).transpose()
        df.columns = [name, 'Datetime']

    except:
        #Hoping this works and just gives me an empty file and not the previous entry.
        print(name + ' failed')
        failed_list.append(name)
        df = pd.DataFrame()
            
    # Write the file
    fname = sys.path[0] + '\data\EO3\\' + year + '_' + name + '.csv'
    df.to_csv(fname, header = False, index = False)
    
    # This step is pretty important, if we're using try/except
    del df
    toc = time.perf_counter()
    print('Wrote ' + name + f" in {toc - tic:0.1f} seconds")
    #df_list.append(df)
    
#full_dataset = pd.concat(df_list, axis = 1)
#full_dataset = full_dataset.loc[:,~full_dataset.columns.duplicated()]
#full_dataset = reduce(lambda  left, right: pd.merge(left, right,on = ['Datetime'], how = 'outer'), df_list)
#full_dataset.set_index('Datetime')

end = time.perf_counter()
print('Done.  ' + f"Total Duration: {(end - start)/60:0.1f} minutes")

Wrote pe32130_eo3 in 22.6 seconds
Wrote fe32212_eo3_op in 19.5 seconds
Wrote pe31018_eo3 in 20.3 seconds
Wrote pe31019_eo3 in 20.2 seconds
Wrote pe31075_eo3 in 5.6 seconds
Wrote pe32665b_eo3 in 16.1 seconds
Wrote pe32665a_eo3 in 15.5 seconds
Wrote le31839_eo3_op in 22.3 seconds
Wrote cae31853_eo3_op in 18.6 seconds
Wrote le32003_eo3_sp in 1.1 seconds
Wrote le32604_eo3 in 22.9 seconds
Wrote fe32550sl_eo3 in 1.1 seconds
Wrote fe31917sl_eo3 in 1.0 seconds
Wrote fe31918sl_eo3 in 1.0 seconds
Wrote le32069_eo3 in 21.9 seconds
Wrote pe33464_eo3_vp in 23.4 seconds
Wrote fe32311_eo3_op in 20.0 seconds
Wrote fe32311_eo3_vp in 25.1 seconds
Wrote le31777sl_eo3 in 1.2 seconds
Wrote le31836sl_eo3 in 0.8 seconds
Wrote le3204_eo3 in 24.3 seconds
Wrote le32066b_eo3 in 21.7 seconds
Wrote pe33464_eo3_op in 19.7 seconds
Wrote ze32071_eo3 in 22.4 seconds
Wrote fe32071_eo3_vp in 19.8 seconds
Wrote ae31877a_eo3 in 6.7 seconds
Wrote fe33341_eo3_vp in 21.2 seconds
Wrote fe33341_eo3_op in 20.0 seconds
Wrote ze3

In [19]:
failed_list

[]

In [11]:
# Build the tag index
tag_index = pd.DataFrame(pi_tags, columns = ['pitag'])
desc_list = [];
eng_units = [];

for pitag in tag_index['pitag']:
    pt = PIPoint.FindPIPoint(piServer, pitag)
    pt.LoadAttributes()
    desc_list.append(pt.GetAttribute('descriptor'))
    eng_units.append(pt.GetAttribute('engunits'))

tag_index['descriptor'] = desc_list
tag_index['eng units'] = eng_units

# Show me the dataframe
tag_index

,pitag,descriptor,eng units
0,PEO3512_EO2,T-EO200 LEAN ABSORBENT PRESS,PSIG
1,CE32267_EO3,Feed Btm Exch Corrosion,MILS
2,E3110CTR_EO3,RX 3110 Coolant Control Temp ROC,DEG F/HR
3,AE33353_EO3,EO Purification Bottoms Pumps P-EO304/308 - EO...,PPM
4,FE32290_EO3,PCC TO EO STRIPPER FLOW CONTROL,MLB/HR
...,...,...,...
2421,TE31689_EO3_T3RSMC_EO3_STRH,T3RSMC TE31689_EO3 CV HI LIM,DEG F
2422,TE31689_EO3_T3RSMC_EO3_STRL,T3RSMC TE31689_EO3 CV LOW LIM,DEG F
2423,FE31140S_I3RSMC1P_EO3_VACT,I3RSMC1P FE31140S MV ACTIVE CONSTRAINT,LL/--/HL
2424,FE31145S_I3RSMC1P_EO3_RECHT,I3RSMC1P FE31145S MV REACH TARGET,CC/HR


In [18]:
failed_list

['fast_te31227_eo3']

In [24]:
# AH OK, I should probably filter out the fast tags
point = PIPoint.FindPIPoint(piServer, 'fast_te31227_eo3')

point.RecordedValues(timerange, AFBoundaryType.Inside, "", False)

PIException: [-11091] Event collection exceeded the maximum allowed
   at OSIsoft.AF.PI.PIException.ConvertAndThrowException(PIServer piServer, Exception ex, String message)
   at OSIsoft.AF.PI.PIPoint.RecordedValues(AFTimeRange timeRange, AFBoundaryType boundaryType, String filterExpression, Boolean includeFilteredValues, Int32 maxCount)

In [12]:
# Write it to CSV
fname = sys.path[0] + '\data\EO3\\' + 'tag_index' + '.csv'
tag_index.to_csv(fname)

In [9]:
#i'm not getting good results with wildcard yet
pt = PIPoint.FindPIPoint(piServer, 'PEO671A_EO2')
# Commands I ran through to get the attribute I'm after
#print(dir(pt))
#print(dir(pt.FindAttributeNames))
pt.FindAttributeNames.__init__()
print(dir(pt.GetAttributes))
pt.FindAttributeNames.__call__('')
# These two are key
pt.GetAttributes.__init__()
pt.GetAttribute('descriptor')

['__call__', '__class__', '__delattr__', '__delete__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__set__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


'K-EO601 DIFF PRESSURE (I-1)'